# SCRIPT PARA ENTRENAR MODELO
Script encargado de reentrenar un modelo previamente entrenado en un dataset gigantesco, para no partir de cero en el entrenamiento y ahorrando tiempo de entrenamiento y obteniendo modelos más eficientes con la libreria *tflite-model-maker*

### IMPORTACIÓN DE LIBRERIAS NECESARIAS
Se definen el conjunto de librerias necesarias utilizadas para reentrenar el modelo con un dataset local, estas son:
* TensorFlow (librería principal de inteligencia artificial) [tf]
* Metodos de TFlite-Model-Maker (librería principal para el reentreanmiento de modelos de inteligencia artificial)  [tflite-model-maker]
* PyPlot (librería mostrar un resultado de prediccion del modelo entrenado)  [plt]
* DateTime (librería para mostrar marcas de tiempo para valorar duracion del entrenamiento)  [datetime]

In [ ]:
import tensorflow as tf
print(tf.__version__)
assert tf.__version__.startswith('2')
from tensorflow.python.client import device_lib
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import matplotlib.pyplot as plt
from datetime import datetime

### COMPROBACIONES INICIALES DE QUE SE DISPONE DE GPU
Debido a que reentrenar estos modelo necesita una capacidad de cómputo razonable y gran tiempo en el proceso, es importante tener de una unidad GPU para la aceleración software, en caso de no encontrar sería recomendable buscar un entorno en la nube para llevar a cabo el reentrenamiento.

In [ ]:
print("||----------------------- LISTADO DE DISPOSITIVOS --------------------||")
print("||------------------------------ RESUMEN -----------------------------||")
print("GPUs: ",len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0],'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus)," GPUs físicas, ", len(logical_gpus)," GPUs lógicas")
    except RuntimeError as error:
        print(error)
print("||--------------------------- CARACTERÍSTICAS ------------------------||")
print(device_lib.list_local_devices())


### DEFINICION DE RUTA DE RAIZ CONJUNTO ENTRENAMIENTO
Definimos la ruta absoluta a la raíz de la que cuelgan las carpetas etiquetadas para el posterior entrenamiento

In [ ]:
# Entrenamiento X salas aumentado
#ruta_dataset = 'D:\TFG\EXPERIMENTOS\DATASET_MUSEO\DatasetXSalasAug' 
# Entrenamiento X vitrinas aumentado
ruta_dataset = 'D:\TFG\EXPERIMENTOS\DATASET_MUSEO\DatasetXVitrinaAug' 

### DIVISION DEL DATASET
Division del dataset de acuerdo con los 3 posibles subconjuntos a crear:
* Conjunto entrenamiento (80% dataset)
* Conjunto validacion (10% dataset)
* Conjunto test (10% dataset)

In [ ]:
datos = DataLoader.from_folder(ruta_dataset)#shuffle=True
datosEntrenamiento,testyValidacion = datos.split(0.8)
datosTest,datosValidacion = testyValidacion.split(0.5)

### ENTRENAMIENTO DEL MODELO
Seleccion del modelo a preentrenar y ajuste de sus hiperparámetros
* ESTRUCTURA METODO image_classifier.create
~~~
train_data 
model_spec ('efficient_lite0')
validation_data (None)
batch_size (None) [64]
epochs (None) [10]
steps_per_epoch (None)
train_whole_model (None)
dropout_rate (None) [0.2]
learning_rate (None) [0.004]
momentum (None) 
shuffle (False)
use_augmentation (False)
use_hub_library (True)[]
warmups_steps (None) [None]
model_dir (None) [tempfile.mkdtemp()]
do_train (True)
~~~
* EXPORTACION DE TFHUB
~~~
modelo_exportado = image_classifier.ModelSpec(uri='https://tfhub.dev/ruta_modelo')
modelo_exportado.input_image_shape = [x,x]
modelo = image_classifier.create(conjuntoEntrenamiento,model_spec=modelo_exportado,...)
# https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/classification/2
~~~
* EXPORTACION DE TFLITE MODEL MAKER
~~~
modelomodelo = image_classifier.create(conjuntoEntrenamiento,model_spec='modelo_tflite',...)
~~~

In [ ]:
# MARCA TIEMPO
now = datetime.now()
print("INICIO =", now)
print("||------------------------- INICIO ENTRENAMIENTO ----------------------||")
modelo = image_classifier.create(datosEntrenamiento,
                                 model_spec='efficientnet_lite4',
                                 validation_data=datosValidacion,
                                 batch_size=32,
                                 epochs=20,
                                 shuffle=True)
# MARCA TIEMPO
now = datetime.now()
print("||-------------------------- FINAL ENTRENAMIENTO -----------------------||")
print("FINAL =", now)

### EVALUACION DEL RESULTADO DEL MODELO RECIÉN GENERADO 
Comprobacion del modelo reentrenado con el conjunto del dataset de test, mediante un extacto de loss y accuracy resultante; con un ejemplo de predicción

In [ ]:
# Funcion para indicar el resultado positivo o negativo en el label
print("||---------------------------- PRUEBA PREDICCION -----------------------||")
def get_label_color(val1,val2):
    if val1 == val2:
        return 'green'
    else:
        return 'red'
    
# Comprobacion de las 100 primeras imagenes de test
plt.figure(figsize=(20,20))
predicciones = modelo.predict_top_k(datosTest)
for i,(image,label) in enumerate(datosTest.gen_dataset().unbatch().take(100)):
    ax = plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(),cmap=plt.cm.gray)
    etiqueta_predecida = predicciones[i][0][0]
    color = get_label_color(etiqueta_predecida,datosTest.index_to_label[label.numpy()])
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' %etiqueta_predecida)
    
plt.show()
print("||-------------------------- EVALUACION DEL MODELO ---------------------||")
modelo.evaluate(datosTest)

### EXPORTACION DEL MODELO AL FORMATO ADECUADO (.tflite)
Exportacion del modelo a formato compatible con Android y evalución del modelo exportado en el formato tflite

In [ ]:
print("||-------------------------- EXPORTACION DEL MODELO --------------------||")
modelo.export(export_dir = 'D:\\TFG\\EXPERIMENTOS\\ModelosMuseo\\efficientnet_lite4_vitrinas_20220705',export_format=ExportFormat.TFLITE)
modelo.export(export_dir = 'D:\\TFG\\EXPERIMENTOS\\ModelosMuseo\\efficientnet_lite4_vitrinas_20220705',export_format=ExportFormat.LABEL)
modelo.export(export_dir = 'D:\\TFG\\EXPERIMENTOS\\ModelosMuseo\\efficientnet_lite4_vitrinas_20220705',export_format=ExportFormat.SAVED_MODEL)
print("||----------------------- EVALUACION DEL MODELO EXPORTADO --------------||")
modelo.evaluate_tflite('D:\\TFG\\EXPERIMENTOS\\ModelosMuseo\\efficientnet_lite4_vitrinas_20220705\\model.tflite',datosTest)

### RESUMEN DE LA ARQUITECTURA

In [ ]:
modelo.summary()

### NOTAS DEL CUADERNO
* ENTRENAMIENTO Nº1
    * TIEMPO INICIO:
    * TIEMPO FINAL:
    * DURACIÓN:
    * INFORMACIÓN ENTRENAMIENTO:
    ~~~
    model_spec ('efficient_lite0') =
    batch_size (None) [64] =
    epochs (None) [10] = 
    steps_per_epoch (None) =
    train_whole_model (None) = 
    dropout_rate (None) [0.2] = 
    learning_rate (None) [0.004] = 
    momentum (None) =
    shuffle (False) =
    use_augmentation (False) =
    use_hub_library (True)[] =
    warmups_steps (None) [None] =
    model_dir (None) [tempfile.mkdtemp()] =
    do_train (True) =
    ~~~